# Proyecto 9: Modelos de Machine Learning para telefonía celular.

# Introducción

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

# Objetivo

Utilizar los diferentes modelos de clasificación para determinar cuál es el que debemos usar en este caso, consideranto tanto precisión como tiempo de ejecución.

# Carga del dataset e importación de librerías

In [1]:
import pandas as pd
#importar modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#importar métricas
from sklearn.metrics import accuracy_score, f1_score
import time

In [2]:
df= pd.read_csv('C:/Users/Oliver/Tripleten/users_behavior.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Primero vamos a establecer cuáles van a ser las características y la variable objetivo, y después separar los datos para entrenamiento y validación de los modelos.

In [3]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [4]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)

Con los parámetros necesarios, podemos empezar la prueba de modelos.

# Árbol de decisión

Vamos a determinar qué profundidad nos da la mejor precisión en este modelo.

In [5]:
#creamos un loop que devuelva el mejor resultado
best_tree_depth = 0
best_accuracy = 0
for depth in range(1,21): 
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train)
    tree_test = model_tree.predict(features_valid)
    tree_score = accuracy_score(tree_test, target_valid)
    if tree_score > best_accuracy:
        best_tree_depth = depth
        best_accuracy= tree_score
print('Mejor profundidad =', best_tree_depth, 'con exactitud de', best_accuracy)

Mejor profundidad = 7 con exactitud de 0.7898009950248757


Dentro de los modelos de árboles de decisión, una profundidad de 7 nos da el mejor resultado, aunque la exactitud parece ser bastante baja, hay que generar los demás modelos para ver cuál brinda los mejores resultados.

# Random Forest

Ahora veremos cuál es la mejor exactitud que podemos obtener con un modelo de random forest, de la mano con el número de estimadores óptimo para maximizar la exactitud.

In [6]:
best_rf_score = 0
best_est= 0
best_depth= 0
for est in range(1,51):
    for depth in range(1,21):
        model_rf = RandomForestClassifier(random_state= 12345, n_estimators=est, max_depth=depth)
        model_rf.fit(features_train, target_train)
        score= model_rf.score(features_valid, target_valid)
        if score > best_rf_score:
            best_rf_score = score
            best_est= est
            best_depth = depth

print('El mejor modelo de Random Forest es con', best_est, 'estimadores, una profundidad máxima de', best_depth, 'y da una exactitud de', best_rf_score)

El mejor modelo de Random Forest es con 19 estimadores, una profundidad máxima de 9 y da una exactitud de 0.8146766169154229


In [7]:
#Variante para f1_score
best_rf_score = 0
best_est = 0
best_depth = 0
for est in range(1, 20):
    for depth in range(1, 21):
        model_rf = RandomForestClassifier(
            random_state=12345, n_estimators=est, max_depth=depth)
        model_rf.fit(features_train, target_train)
        predict_valid = model_rf.predict(features_valid)
        score = f1_score(target_valid, predict_valid)
        if score > best_rf_score:
            best_rf_score = score
            best_est = est
            best_depth = depth
    
print('El mejor modelo de Random Forest es con', best_est, 'estimadores, una profundidad máxima de', best_depth, 'y tiene un valor f1 de', best_rf_score)

El mejor modelo de Random Forest es con 19 estimadores, una profundidad máxima de 9 y tiene un valor f1 de 0.6227848101265823


Notamos que obtenemos una puntuación ligeramente mejor con este modelo, aunque para obtenerla necesitamos aplicar un gran número de estimadores, lo que puede significar un gran tiempo de ejecución si se traduce a un dataset más grande. Sin embargo, con este conjunto pequeño el tiempo podría no ser considerable.

# Regresión logística

Por último, haremos una prueba de regresión logística para determinar si este modelo es más preciso que los otros dos.

In [8]:
model_log= LogisticRegression(random_state=12345, solver='liblinear')
model_log.fit(features_train, target_train)
score_log= model_log.score(features_valid, target_valid)
print('Precisión del modelo de regresión logística:', score_log)

Precisión del modelo de regresión logística: 0.7027363184079602


Este modelo ha tenido la menor precisión de los 3, así que lo descartaremos como opción.

# Tiempo de ejecución

El árbol de decisión y random forest tuvieron resultados similares, ahora hay que comprobar el tiempo de ejecución para determinar si es recomendable usar random forest o si por ese pequeño porcentaje extra, no lo vale. Para el árbol de decisión usaremos la profundidad máxima de 7 y para random forest usaremos 60 estimadores.
Nos basaremos en el tiempo que tarda cada uno en "predecir" los resultados de todo el dataset.

In [9]:
#usaremos el módulo time para medir el tiempo de ejecución
#primero mediremos el tiempo que toma en predecir el árbol de decisión
start_tree = time.time()

tree= DecisionTreeClassifier(random_state=12345, max_depth=7)
tree.fit(features, target)
tree_predictions= tree.predict(features)

end_tree = time.time()
print('Tiempo de ejecución:', end_tree - start_tree)

Tiempo de ejecución: 0.006507158279418945


In [10]:
#ahora haremos la misma prueba con el modelo de random forest 
start_rf = time.time()

forest = RandomForestClassifier(random_state=12345, n_estimators = 60)
forest.fit(features, target)
rf_predictions = forest.predict(features)

end_rf = time.time()
print('Tiempo de ejecución:', end_rf - start_rf)

Tiempo de ejecución: 0.23861265182495117


In [11]:
%%time

forest = RandomForestClassifier(random_state=12345, n_estimators = 60)
forest.fit(features, target)
rf_predictions = forest.predict(features)


CPU times: total: 234 ms
Wall time: 235 ms


In [12]:
%%time
tree = DecisionTreeClassifier(random_state=12345, max_depth=7)
tree.fit(features, target)
tree_predictions = tree.predict(features)

CPU times: total: 0 ns
Wall time: 6.07 ms


Como era de esperarse, el tiempo de ejecución es bastante elevado, y con menor cantidad de estimadores la diferencia en precisión del modelo es aún menor y sigue tardando considerablemente más, así que concluimos que el mejor modelo a utilizar será el árbol de decisión.

# Conclusión 

Después de evaluar los 3 diferentes modelos, concluimos que el óptimo para este problema es un árbol de decisión, ya que el random forest tiene una precisión bastante similar pero tarda mucho más en realizar las predicciones, y aunque en este caso el tiempo sigue siendo muy poco, al trasladarlo a una base de datos mayor el tiempo se dispararía y con muy poco beneficio en precisión.